In [58]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import plotly.express as px
import json
import plotly.graph_objects as go
from geopy.geocoders import Nominatim
from shapely.geometry import Point, Polygon
from area import area
from sklearn import neighbors

%matplotlib inline

In [59]:
'''
postmates = pd.read_csv('/Users/baileyrusso/PycharmProjects/GrubHubScraper/postmates.csv')
postmates = postmates.loc[:, ~postmates.columns.str.contains('^Unnamed')]
postmates['Address_Coordinates'] = list(map(lambda row: str(row).replace("(","").replace(")","").split(","), postmates['Address_Coordinates']))
postmates['Address_Coordinates'] = list(map(lambda row: tuple([float(i) for i in row if pd.notnull(i)]) , postmates['Address_Coordinates']))
postmates[['Latitude', 'Longitude']] = pd.DataFrame(postmates['Address_Coordinates'].tolist(), index=postmates.index)
postmates['Favorites'] = postmates['Favorites'].fillna(0)
'''

'\npostmates = pd.read_csv(\'/Users/baileyrusso/PycharmProjects/GrubHubScraper/postmates.csv\')\npostmates = postmates.loc[:, ~postmates.columns.str.contains(\'^Unnamed\')]\npostmates[\'Address_Coordinates\'] = list(map(lambda row: str(row).replace("(","").replace(")","").split(","), postmates[\'Address_Coordinates\']))\npostmates[\'Address_Coordinates\'] = list(map(lambda row: tuple([float(i) for i in row if pd.notnull(i)]) , postmates[\'Address_Coordinates\']))\npostmates[[\'Latitude\', \'Longitude\']] = pd.DataFrame(postmates[\'Address_Coordinates\'].tolist(), index=postmates.index)\npostmates[\'Favorites\'] = postmates[\'Favorites\'].fillna(0)\n'

# Plotting Restaurant Coordinates

In [60]:
'''
crs = {'init':'epsg:2263'}
geometry = [Point(addr[1], addr[0]) for addr in postmates['Address_Coordinates']]
geo_df = gpd.GeoDataFrame(postmates, crs=crs, geometry=geometry)
geo_df
'''

"\ncrs = {'init':'epsg:2263'}\ngeometry = [Point(addr[1], addr[0]) for addr in postmates['Address_Coordinates']]\ngeo_df = gpd.GeoDataFrame(postmates, crs=crs, geometry=geometry)\ngeo_df\n"

In [61]:
'''
street_map = gpd.read_file('pluto/pluto.shp')
fig, ax = plt.subplots(figsize=(15,15))
street_map.plot(ax = ax, alpha=0.4, color='gray')
geo_df.plot(ax=ax, markersize=20,color='blue',marker='o')
'''

"\nstreet_map = gpd.read_file('pluto/pluto.shp')\nfig, ax = plt.subplots(figsize=(15,15))\nstreet_map.plot(ax = ax, alpha=0.4, color='gray')\ngeo_df.plot(ax=ax, markersize=20,color='blue',marker='o')\n"

In [62]:
#nycmapURL = '/Users/baileyrusso/PycharmProjects/GrubHubScraper/pluto.geojson'

In [63]:
#all_data_diffq = (postmates["Favorites"].max() - postmates["Favorites"].min()) / 16
postmates_grouped = postmates.groupby(['Name','Category']).mean().reset_index()
fav_scaled = (postmates_grouped["Favorites"].max() - postmates_grouped["Favorites"].min()) / 16
postmates_grouped["scale"] = (postmates_grouped["Favorites"] - postmates_grouped["Favorites"].min()) / fav_scaled + 1



In [64]:
postmates_grouped

,Name,Category,Favorites,MenuItemPrice,Latitude,Longitude,scale
0,Aahar Indian Cuisine,INDIAN,57.0,11.332184,40.713308,-74.008042,1.276364
1,Al Horno Lean Mexican Kitchen,MEXICAN,0.0,8.540650,40.704702,-74.008857,1.000000
2,Ali Baba Organic Marketplace,GROCERY,159.0,8.891416,40.713719,-73.998778,1.770909
3,Alimama Tea,DESSERT,102.0,4.365854,40.715801,-73.999247,1.494545
4,Antique Garage Soho,MEDITERRANEAN,0.0,25.446809,40.721481,-74.001373,1.000000
...,...,...,...,...,...,...,...
167,Yaya Tea Garden,TEA,104.0,3.906098,40.716202,-73.994952,1.504242
168,Yi Fang Taiwan Fruit Tea,TEA,0.0,6.208333,40.715352,-73.998268,1.000000
169,by CHLOE.,VEGAN,264.0,6.872368,40.702526,-73.985988,2.280000
170,sweetgreen,VEGETARIAN,76.0,11.915714,40.715464,-74.008865,1.368485


In [65]:
token = 'pk.eyJ1IjoiYmJydXNzbzExNSIsImEiOiJja29qY2x4Z2wwMnk3MnBvNzRldXo2a2J2In0.6zgZNfkLxe6DylEBbMglZA'

fig = px.scatter_mapbox(postmates_grouped, lat="Latitude", lon="Longitude", 
                        size="scale", hover_name="Name", color="Category", hover_data=["Favorites"], zoom=10, height=300)
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
fig.update_traces(marker={'sizemin':4})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()
